# Nutrient optimization

Demo code to generate (unlimited varieties of) nutritionally complete diets.

I apply default filtering of gross things like baby food, but foods can be filtered to meet any dietary preference.

Current default is to,

minimize *total calories* 

subjct to nutritional completeness

Any other nutritional optimization is possible too: maximize protein, minimize carbs, etc.

Technique: Nested optimization. 
 
Innermost set is a basket of foods. To find the amount of each food that meets nutritional requirements I use an LP solver

Outer optimization is genetic algorithm. It generates baskets of foods with items that tend to perform well together.
 

## Import custom helper functions

* def load_data():
* def do_clust(N,lim,req,nut):
* def evaluate(individual, nut,limt,reqd):
* def makeclusters(nclust,limt,reqd,nutrients ):
* def InitPopulation( pcls, ind_init,nfood, nclust, nseed,clust):
* def generate_ssdum(random, args):

In [1]:
import sys
print(sys.path)
from lib.libraries import *

['/home/pedwards/documents/SpartanSupper/git', '/home/pedwards/.conda/envs/spartansupper/lib/python37.zip', '/home/pedwards/.conda/envs/spartansupper/lib/python3.7', '/home/pedwards/.conda/envs/spartansupper/lib/python3.7/lib-dynload', '', '/home/pedwards/.conda/envs/spartansupper/lib/python3.7/site-packages', '/home/pedwards/.conda/envs/spartansupper/lib/python3.7/site-packages/IPython/extensions', '/home/pedwards/.ipython']


## Import external libraries

Most are standard, but we want the glpk solver for cvxopt, which requires the following,
```
$ sudo apt-get install libglpk-dev
$ sudo CVXOPT_BUILD_GLPK=1 pip install cvxopt
```

In [2]:
from os import path

import pickle
import pandas
import numpy
from deap import base, creator, tools, algorithms
from sklearn.preprocessing import normalize
from cvxopt import matrix, solvers # an alternative linear programming library
solvers.options['show_progress'] = False

from sklearn.cluster import KMeans
import random
from time import time




pandas.set_option('display.max_rows', None)
pandas.set_option('display.max_columns', None)

## Load food & nutrients from database

In [ ]:
numpy.random.seed(seed=333)
(nutrients,reqd,limt,food_desc,nutrient_desc)=load_data()
print( '[*] Loaded %d foods from database' % nutrients.shape[0] )

## Cluster

Observation: the optimization converges faster (and to lower minima) if provided a "seed" population with random baskets of *diverse* foods

Technique: I use a kmeans to find clusters of food types then sample from them (with a multinomial dist)

In [11]:
if not path.exists('clust.pkl'):
    Nclust=15
    clust=makeclusters(Nclust,limt,reqd,nutrients )
    pickle.dump( clust, open( "clust.pkl", "wb" ) )
else:
    clust=pickle.load( open( "clust.pkl", "rb" ) )
    print( '[*] Found pickle file with %d clusters and %d foods' % (clust.max()+1,len(clust)) )

[*] Loaded 4492 foods from database
[*] Found pickle file with 15 clusters and 4492 foods


## Genetic algorithm

This is the outermost optimization layer (the inner optim is in `evaluate_ss()`)

Todo: I don't know why generator makes "extra" values in the vector. The non-integer values are ignored anyway but that is strange.

Todo: play with early stopping. no need to keep going after imporvement has ceased or slowed.

In [4]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin) # an individual comprises a list (of food IDs)
N_FOODS=6
Nseed=500
NT_DIM=nutrients.shape[0]
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_foodid", random.randrange, NT_DIM)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_foodid, N_FOODS)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=NT_DIM, indpb=0.1)
#toolbox.register("select", tools.selBest, k=3)
toolbox.register("select", tools.selTournament, tournsize=10)
toolbox.register("evaluate", evaluate, nut=nutrients,limt=limt,reqd=reqd)

# used to make a seed population (only) ; per: https://deap.readthedocs.io/en/master/tutorials/basic/part1.html?highlight=seeding#seeding-a-population
toolbox.register("population_guess", InitPopulation, list, creator.Individual, N_FOODS,Nclust,Nseed,limt,reqd,nutrients )

stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("median", numpy.median)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

In [5]:
%%time

pop = toolbox.population(n=300) # totally random initial population
#pop = toolbox.population_guess()
pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=50, 
                                   stats=stats, verbose=True)

gen	nevals	median	std        	min   	max  
0  	300   	9e+09 	2.57563e+09	1701.9	9e+09
1  	181   	9e+09 	4.47432e+09	1685.59	9e+09
2  	173   	2603.48	3.84375e+09	1200.89	9e+09
3  	181   	1701.9 	3.3541e+09 	1200.89	9e+09
4  	178   	1396.96	3.05941e+09	1200.89	9e+09
5  	190   	1204.95	2.48747e+09	1091.26	9e+09
6  	198   	1200.89	1.26e+09   	1091.26	9e+09
7  	174   	1200.67	8.95489e+08	1091.26	9e+09
8  	160   	1091.26	1.15217e+09	1091.26	9e+09
9  	171   	1091.26	1.26e+09   	1075.86	9e+09
10 	165   	1091.26	1.26e+09   	956.144	9e+09
11 	173   	1091.26	1.26e+09   	920.993	9e+09
12 	171   	1075.86	1.89831e+09	920.993	9e+09
13 	172   	956.144	1.03228e+09	893.36 	9e+09
14 	175   	920.993	1.69148e+09	893.36 	9e+09
15 	205   	920.993	1.03228e+09	885.903	9e+09
16 	165   	893.36 	5.18748e+08	885.903	9e+09
17 	170   	893.36 	1.03228e+09	873.495	9e+09
18 	186   	885.903	1.53528e+09	873.495	9e+09
19 	185   	885.903	1.61555e+09	873.495	9e+09
20 	189   	873.495	1.9615e+09 	792.674	9e+09
21 	193   	873.

In [6]:
best=tools.selBest(pop, k=1)
best=best[0]
evaluate(best, nut=nutrients,limt=limt,reqd=reqd)
nt=nutrients.iloc[best,:]
c = matrix(numpy.repeat(1.0,nt.shape[0]))
np_G= numpy.concatenate(
                        (   nt.transpose().values, 
                            nt.transpose().multiply(-1.0).values,
                            numpy.diag(numpy.repeat(-1,nt.shape[0])) 
                        )
                       ).astype(numpy.double) 
G = matrix( np_G ) 
h = matrix( numpy.concatenate( (
                limt.values, 
                reqd.multiply(-1.0).values, 
                numpy.repeat(0.0,nt.shape[0])
            ) ).astype(numpy.double) )    
o = solvers.lp(c, G, h, solver='glpk')
food_amounts=numpy.array(o['x'])[:,0]


## Print the best "diet"

This is the food and corresponding amount to eat (in grams, sorry bud!). The idea is if you eat all this in a day you have the nutrients you need for the day.

Todo: this would be better if split up into recipes. 

In [7]:
final_foods= [ best[i] for i in range(len(food_amounts)) if food_amounts[i]>1e-6 ] # select those with non-trivial amounts
final_food_amounts= food_amounts[ food_amounts>1e-6 ]

nt=nutrients.iloc[final_foods,:]
df1= nt.join(food_desc).loc[:,['long_desc']] #food_desc.iloc[final_foods]
df2=pandas.DataFrame(final_food_amounts*100,index=df1.index,columns=["amount"])
df3=pandas.DataFrame(nt.loc[:,208].values * df2.loc[:,'amount'].values/100 ,columns=['calories'], index=df2.index)
diet_table=df1.join(df2).join(df3)

In [8]:
diet_table

,long_desc,amount,calories
food_id,,,
11092,"Broccoli, frozen, chopped, unprepared",919.590880,239.093629
17425,"Veal, leg, top round, cap off, cutlet, boneles...",103.463678,156.230153
13322,"Beef, variety meats and by-products, heart, co...",102.432495,169.013617
11733,"Beans, snap, yellow, frozen, cooked, boiled, d...",97.607378,25.377918
11896,"Winged bean, immature seeds, cooked, boiled, d...",370.282043,137.004356


In [11]:
nutrient_totals=pandas.DataFrame( ( 
                    numpy.dot( numpy.transpose(final_food_amounts), nt.values),
                    reqd,
                    limt
                  ), index=['Total','Amount required','Amount limit'], columns=nt.columns).transpose()

In [12]:
A=nt.join(df1).set_index('long_desc').transpose() * final_food_amounts
nutrient_report=A.join(nutrient_desc).join(nutrient_totals).set_index('name')
nutrient_report

,"Broccoli, frozen, chopped, unprepared","Veal, leg, top round, cap off, cutlet, boneless, cooked, grilled","Beef, variety meats and by-products, heart, cooked, simmered","Beans, snap, yellow, frozen, cooked, boiled, drained, with salt","Winged bean, immature seeds, cooked, boiled, drained, with salt",Total,Amount required,Amount limit
name,,,,,,,,
Protein,25.840502,32.994564,29.172773,1.454350,19.661978,109.124170,53.459999,1.000000e+10
Total lipid (fat),2.666813,2.721095,4.845057,0.165933,2.443862,12.842759,0.000000,1.000000e+10
"Carbohydrate, by difference",43.956444,0.000000,0.153649,5.719793,11.886054,61.715941,0.000000,1.000000e+10
Ash,6.069300,1.500223,0.993595,1.034638,2.629003,12.226759,0.000000,1.000000e+10
Energy,239.093628,156.230148,169.013611,25.377918,137.004364,726.719673,0.000000,1.000000e+10
Starch,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+10
Sucrose,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+10
Glucose (dextrose),6.896932,0.000000,0.000000,0.000000,0.000000,6.896932,0.000000,1.000000e+10
Fructose,7.632604,0.000000,0.000000,0.000000,0.000000,7.632604,0.000000,1.000000e+10
